## /SEMANTIX_ACADEMY
#### /PROJETO_FINAL_SPARK // NÍVEL AVANÇADO // CAMPANHA NACIONAL DE VACINAÇÃO CONTRA CONVID-19
    /ALUNO: FELIPE CHAGAS DE SOUZA
    /E-MAIL: souza.chagas.felipe@gmail.com
    /TREINAMENTO: BIG DATA ENGINNER // 2022

In [4]:
import os
import requests
import logging
import json
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Row

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('LOGGER')
logger.info('Imports e setup do logger realizados!')

INFO:LOGGER:Imports e setup do logger realizados!


In [17]:
def data_collector():
    headers = {'Accept': 'application/json'}
    url = 'https://imunizacao-es.saude.gov.br/_search'
    user = 'imunizacao_public'
    password = 'qlto5t&7r_@+#Tlstigi'
    data = {'scroll':'1m','size':'10000'}

    return requests.post(url, params=data, headers=headers, auth=(user, password))
    logger.info("Request a API realizado com sucesso")

In [18]:
response_api_covid_br = data_collector()

df_api_covid_br = spark.createDataFrame([Row(**obj['_source']) for obj in response_api_covid_br.json()['hits']['hits']])
logger.info("Response_api_covid_br carregada no Dataframe df_api_covid_br")

INFO:LOGGER:Response_api_covid_br carregada no Dataframe df_api_covid_br


In [21]:
df_api_covid_br.printSchema()

root
 |-- @timestamp: string (nullable = true)
 |-- @version: string (nullable = true)
 |-- data_importacao_datalake: string (nullable = true)
 |-- data_importacao_rnds: string (nullable = true)
 |-- document_id: string (nullable = true)
 |-- dt_deleted: string (nullable = true)
 |-- estabelecimento_municipio_codigo: string (nullable = true)
 |-- estabelecimento_municipio_nome: string (nullable = true)
 |-- estabelecimento_razaoSocial: string (nullable = true)
 |-- estabelecimento_uf: string (nullable = true)
 |-- estabelecimento_valor: string (nullable = true)
 |-- estalecimento_noFantasia: string (nullable = true)
 |-- id_sistema_origem: string (nullable = true)
 |-- paciente_dataNascimento: string (nullable = true)
 |-- paciente_endereco_cep: string (nullable = true)
 |-- paciente_endereco_coIbgeMunicipio: string (nullable = true)
 |-- paciente_endereco_coPais: string (nullable = true)
 |-- paciente_endereco_nmMunicipio: string (nullable = true)
 |-- paciente_endereco_nmPais: string

In [42]:
view_idade_media_uf = df_api_covid_br.groupBy(col("estabelecimento_uf").alias("sigla_uf")\
                                     , col("estabelecimento_municipio_nome").alias("nome_municipio"))\
                .agg(format_number(avg(col("paciente_idade")),2).alias("idade_media_vacinado")).sort(asc("sigla_uf")\
                                                                                                     , asc("nome_municipio"))

print("/IDADE_MEDIA_POR_UF_MUNICIPIO")
view_idade_media_uf.show(truncate=False)

/IDADE_MEDIA_POR_UF_MUNICIPIO
+--------+-------------------+--------------------+
|sigla_uf|nome_municipio     |idade_media_vacinado|
+--------+-------------------+--------------------+
|null    |null               |26.50               |
|AC      |FEIJO              |20.75               |
|AC      |MANOEL URBANO      |49.00               |
|AC      |PORTO ACRE         |14.00               |
|AC      |PORTO WALTER       |27.00               |
|AC      |RIO BRANCO         |32.59               |
|AC      |SANTA ROSA DO PURUS|12.00               |
|AC      |SENA MADUREIRA     |11.67               |
|AC      |SENADOR GUIOMARD   |8.00                |
|AC      |XAPURI             |7.00                |
|AL      |ARAPIRACA          |25.33               |
|AL      |BATALHA            |27.00               |
|AL      |BELEM              |21.00               |
|AL      |BOCA DA MATA       |55.00               |
|AL      |CAMPESTRE          |40.00               |
|AL      |CHA PRETA          |36.0

In [47]:
view_qtde_aplicacao_vacina = df_api_covid_br.groupBy(col("vacina_fabricante_nome").alias("vacina"))\
                                            .agg(count(col("vacina_descricao_dose"))\
                                            .alias("qtde")).sort(asc("vacina"))

print("/QTDE_APLICACAO_VACINA")
view_qtde_aplicacao_vacina.show(truncate=False)

/QTDE_APLICACAO_VACINA
+----------------------+----+
|vacina                |qtde|
+----------------------+----+
|ASTRAZENECA           |17  |
|ASTRAZENECA/FIOCRUZ   |2149|
|JANSSEN               |2348|
|PFIZER                |3051|
|Pendente Identificação|1408|
|SINOVAC/BUTANTAN      |1027|
+----------------------+----+



In [51]:
view_qtde_aplicacao_vacina_uf = df_api_covid_br.groupBy( col("estabelecimento_uf").alias("sigla_uf")\
                                                    , col("vacina_fabricante_nome").alias("vacina"))\
                                            .agg(count(col("vacina_descricao_dose"))\
                                            .alias("qtde")).sort(asc("sigla_uf"))

print("/QTDE_APLICACAO_VACINA_POR_UF")
view_qtde_aplicacao_vacina_uf.show(truncate=False)

/QTDE_APLICACAO_VACINA_POR_UF
+--------+----------------------+----+
|sigla_uf|vacina                |qtde|
+--------+----------------------+----+
|null    |PFIZER                |2   |
|AC      |PFIZER                |20  |
|AC      |JANSSEN               |1   |
|AC      |SINOVAC/BUTANTAN      |5   |
|AC      |Pendente Identificação|9   |
|AL      |Pendente Identificação|38  |
|AL      |SINOVAC/BUTANTAN      |5   |
|AL      |PFIZER                |94  |
|AL      |JANSSEN               |2   |
|AM      |ASTRAZENECA           |2   |
|AM      |Pendente Identificação|24  |
|AM      |JANSSEN               |10  |
|AM      |PFIZER                |128 |
|AM      |SINOVAC/BUTANTAN      |35  |
|AM      |ASTRAZENECA/FIOCRUZ   |35  |
|AP      |PFIZER                |17  |
|AP      |SINOVAC/BUTANTAN      |6   |
|AP      |ASTRAZENECA/FIOCRUZ   |1   |
|AP      |Pendente Identificação|9   |
|BA      |JANSSEN               |147 |
+--------+----------------------+----+
only showing top 20 rows



In [81]:
view_idade_media_vacina = df_api_covid_br.groupBy(col("vacina_fabricante_nome").alias("vacina"))\
                .agg(format_number(avg(col("paciente_idade")),2).alias("idade_media_vacinado")).sort(asc("vacina"))

print("/IDADE_MEDIA_POR_VACINA")
view_idade_media_vacina.show(truncate=False)

/IDADE_MEDIA_POR_VACINA
+----------------------+--------------------+
|vacina                |idade_media_vacinado|
+----------------------+--------------------+
|ASTRAZENECA           |54.18               |
|ASTRAZENECA/FIOCRUZ   |42.43               |
|JANSSEN               |52.79               |
|PFIZER                |42.14               |
|Pendente Identificação|8.36                |
|SINOVAC/BUTANTAN      |21.83               |
+----------------------+--------------------+



In [77]:
view_balanco_doses = df_api_covid_br.groupBy(col("vacina_descricao_dose").alias("descricao_dose"))\
        .agg(countDistinct(col("paciente_id")).alias("qtde"),\
            ).sort(asc("descricao_dose"))

print("/BALANÇO_QTDE_POR_DOSE")
view_balanco_doses.show(truncate=False)

/BALANÇO_QTDE_POR_DOSE
+--------------+----+
|descricao_dose|qtde|
+--------------+----+
|1ª Dose       |859 |
|1º Reforço    |272 |
|2ª Dose       |2204|
|2º Reforço    |996 |
|3ª Dose       |117 |
|4ª Dose       |293 |
|Dose          |37  |
|Dose Adicional|329 |
|Reforço       |4892|
|Única         |1   |
+--------------+----+



In [80]:
view_top_data_aplicacao = df_api_covid_br.groupBy(date_format(col("vacina_dataAplicacao"), "dd/MM/yyyy").alias("data"))\
        .agg(countDistinct(col("paciente_id")).alias("qtde"),\
            ).sort(desc("qtde"))

print("/TOP_10_DATA_APLICACAO")
view_top_data_aplicacao.show(10, truncate=False)

/TOP_10_DATA_APLICACAO
+----------+----+
|data      |qtde|
+----------+----+
|20/04/2022|2193|
|12/04/2022|1106|
|19/04/2022|610 |
|13/04/2022|529 |
|11/04/2022|341 |
|18/04/2022|266 |
|21/04/2022|221 |
|08/04/2022|200 |
|07/04/2022|158 |
|01/04/2022|135 |
+----------+----+
only showing top 10 rows



In [83]:
logger.info("Salvando as views como tabelas HIVE...")

view_idade_media_uf.write.saveAsTable("tb_idade_media_uf", mode="overwrite")
view_qtde_aplicacao_vacina.write.saveAsTable("tb_qtde_aplicacao_vacina", mode="overwrite")
view_qtde_aplicacao_vacina_uf.write.saveAsTable("tb_qtde_aplicacao_vacina_uf", mode="overwrite")
view_idade_media_vacina.write.saveAsTable("tb_idade_media_vacina", mode="overwrite")
view_balanco_doses.write.saveAsTable("tb_balanco_doses", mode="overwrite")
view_top_data_aplicacao.write.saveAsTable("tb_top_data_aplicacao", mode="overwrite")


!hdfs dfs -ls /user/hive/warehouse/tb_idade_media_uf | head -1
!hdfs dfs -ls /user/hive/warehouse/tb_qtde_aplicacao_vacina | head -1
!hdfs dfs -ls /user/hive/warehouse/tb_qtde_aplicacao_vacina_uf | head -1
!hdfs dfs -ls /user/hive/warehouse/tb_idade_media_vacina | head -1
!hdfs dfs -ls /user/hive/warehouse/tb_balanco_doses | head -1
!hdfs dfs -ls /user/hive/warehouse/tb_top_data_aplicacao | head -1

logger.info("Views foram salvas com sucesso!")

INFO:LOGGER:Salvando as views como tabelas HIVE...


Found 7 items
Found 29 items
Found 29 items
Found 7 items
Found 11 items
Found 64 items


INFO:LOGGER:Views foram salvas com sucesso!


### /COLETA DE TODOS OS DADOS ATRAVÉS DA API:
#### LAÇO PARA COLETAR TODOS OS REGISTROS PELA API ATRAVÉS DO SCROLL_ID E HITS RESTANTES
##### PRECISA DE OTIMIZAÇÃO PARA REALIZAR A EXECUÇÃO, TALVEZ CRIAR METÓDO DE CACHE/TEMP TABLE

```
CÓDIGO ABAIXO:
```

In [ ]:
def get_scroll_id(url, headers, user, password, params):

    response_scroll_id = requests.get(url, params=params, headers=headers, auth=(user, password))
    
    if response_scroll_id.status_code != 200:
        raise Exception(response_scroll_id.status_code, response_scroll_id.text)
        
    return response_scroll_id.json()['_scroll_id']
    
def total_hits(url, headers, user, password, params):
    
    response_total_hits = requests.get(url, params=params, headers=headers, auth=(user, password))
    
    if response_total_hits.status_code != 200:
        raise Exception(response_total_hits.status_code, response_total_hits.text)
    
    return response_total_hits.json()['hits']['total']['value']
    
    
def connect_api(url, headers, user, password, params):
    
    response = requests.post(url, params=params, headers=headers, auth=(user, password))
    print(response.status_code)
    
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def data_collector(url, url_scroll_id, headers, data, user, password, params, params_scroll_id, total_hits, scroll_id=""):
    scroll_limit = 31
    rows = []
    #total_hits = total_hits(url, headers, user, password, params)
    
    if scroll_id:
        full_params = {'scroll':'1m', 'scroll_id': scroll_id}
    else:
        full_params = {'scroll':'1m', 'scroll_id': get_scroll_id(url_scroll_id, headers, user, password, params_scroll_id)}

    data = connect_api(url, headers, user, password, full_params)

    for page_number in range(scroll_limit): #range(total_hits)

        rows = requests.post(url, params=full_params, headers=headers, auth=(user, password))
 
    return rows


def main():
    headers = {'Accept': 'application/json'}
    url = 'https://imunizacao-es.saude.gov.br/_search/scroll'
    user = 'imunizacao_public'
    password = 'qlto5t&7r_@+#Tlstigi'
    params = {'scroll':'1m', 'scroll_id': get_scroll_id()}

    for json_response in paginate(url, headers, data, user, password, params):
       print(json.dumps(json_response, indent=4, sort_keys=True))


def execute_data_collector():
    url_basic = 'https://imunizacao-es.saude.gov.br/_search'
    params_basic = {'scroll':'1m'}
    headers = {'Accept': 'application/json'}
    url_scroll = 'https://imunizacao-es.saude.gov.br/_search/scroll'
    user = 'imunizacao_public'
    password = 'qlto5t&7r_@+#Tlstigi'
    params = {'scroll':'1m', 'scroll_id': get_scroll_id(url_basic, headers, user, password, params_basic)}
    
    return data_collector(url_scroll, url_basic, headers, data, user, password, params, params_basic, total_hits(url_basic, headers, user, password, params_basic))

response_rows_api = execute_data_collector()

df_covid_br_api = spark.createDataFrame([Row(**obj['_source']) for obj in response_rows_api.json()['hits']['hits']])

df_covid_br_api.count()
df_covid_br_api.printSchema()